# Step 4: Evaluate the application

## 1. Run the app on our evaluation dataset

In [1]:
%run 01-llm-app-setup.ipynb

Context: ['Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.', 'Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) o

In [2]:
import pandas as pd

gen_dataset = pd.read_csv('generated_qa.csv')

In [9]:
gen_dataset["answer"] = None
gen_dataset["contexts"] = None

for idx, item in gen_dataset.iloc[:2].iterrows():
    answer = generative_block.invoke({"question":item.question, "context": concat_string([item.ground_truth_context])})
    gen_dataset.at[idx, "contexts"] = [item.ground_truth_context] # Keep this static to evaluate generation only
    gen_dataset.at[idx, "answer"] = answer


In [10]:
gen_dataset.iloc[:2]

,question,ground_truth,ground_truth_context,answer,contexts
0,What is the core controller of the autonomous ...,LLM (large language model),LLM Powered Autonomous Agents\n \nDate: Jun...,The core controller of the autonomous agents d...,[LLM Powered Autonomous Agents\n \nDate: Ju...
1,What is considered as utilizing the short-term...,"In-context learning, as seen in Prompt Enginee...",Memory\n\nShort-term memory: I would consider ...,Utilizing in-context learning for prompt engin...,[Memory\n\nShort-term memory: I would consider...


## 2. Run evaluation

This might take a few minutes. We call our pre-defined evaluation functions and also run ragas on our dataset.

In [11]:
%run 03-metrics-definition.ipynb

In [12]:
results_lst = []
 
for idx, row in gen_dataset.iloc[:2].iterrows(): # Subsetting to make it go faster
    custom_eval_results = {
        "rouge": rouge(row["ground_truth"], row["answer"]),
        "levenshtein": levenshtein(row["ground_truth"], row["answer"]),
        "toxicity": toxicity(row["ground_truth"], row["answer"]),
    }

    ragas_eval_results = evaluate_w_ragas(row)
    results_lst.append(custom_eval_results | ragas_eval_results)


results_df = pd.DataFrame(results_lst)



Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Evaluating:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
results_df

,context_correctness,ground_truth_context_rank,context_rougel_score,context_precision,faithfulness,answer_correctness
0,True,0,1.0,1.0,1.0,0.592283
1,True,0,1.0,1.0,1.0,0.468771


And that's it! We can aggregate these metrics to get a single number for each of them and we have a good evaluation of our model.